# Introduction to Tensorflow
Estimated time: 1:30 hours

## Importing necessary libraries

In [2]:
import tensorflow as tf
import numpy as np

# Checking if a GPU is available
try:
    assert tf.test.is_gpu_available()
except:
    raise AssertionError("A GPU was not detected. If your computer does not have a GPU, ignore this message")

## Good old Python vs TensorFlow

In [3]:
tf.reset_default_graph()

mat_size = 5000
def numpy_dot(a,b):
    return np.dot(a,b)

def tf_dot(a,b):
    tf_a = tf.placeholder(shape=[mat_size, mat_size], dtype=tf.float32, name='a')
    tf_b = tf.placeholder(shape=[mat_size, mat_size], dtype=tf.float32, name='b')
    with tf.Session() as sess:
        return sess.run(tf.matmul(tf_a,tf_b), feed_dict={tf_a: a, tf_b: b})
    
# Python doing a dot product
a = np.random.rand(mat_size, mat_size)
b = np.random.rand(mat_size, mat_size)

np_time = %timeit -o numpy_dot(a,b)
tf_time = %timeit -o tf_dot(a,b)

print("Numpy took {:.1f}x times more time than TensorFlow".format(np_time.best*1.0/tf_time.best))


3.69 s ± 501 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
614 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Numpy took 5.7x times more time than TensorFlow


## Numpy is very slow (for large matrices)

Not just matrix multiplication, but also other operations like convolution which are ubiquotous in neural networks. This is where TensorFlow fits in. Tensorflow has highly GPU-optimized CUDA kernels for these operations which allows TensorFlow to exploit the parallel computation power of GPUs. Where a very good CPU might be able to run 128 threads (vectorized and hyper-threaded) simultaneously, NVIDIA GTX 1080 has 2560 cores.

matrix multiplication parallel image

## Why use TensorFlow?

> TensorFlow is an end-to-end open source platform for machine learning. It has a comprehensive, flexible ecosystem of tools, libraries and community resources that lets researchers push the state-of-the-art in ML and developers easily build and deploy ML powered applications.

* Efficiently leverages the explicit parallel nature of GPUs to perform parallelizable computations quickly
* Has almost all the high-level functions you will need to implement simple or complex neural network
* Provides automatic differentiation so you don't have to write the backward pass of neural networks


## Diving head-first into TensorFlow

Here we will be diving into TensorFlow specifics

## 1.1. Placeholders, variables, tensors and operations

Here you will be learning about placeholders, variables, tensors and operations.
* Placeholder: A type of tensor where data is fed at run time
* Variable: A type of tensor which is mutable, of which value can be changed later
* Tensor: An immutable tensor that cannot be changed ones initialized
* Operation: Performs transformations on various tensors to produce new outputs

In [3]:
tf.reset_default_graph()
a = tf.placeholder(name='a', shape=[5,10], dtype=tf.float32)
print("\'{}\' is a {}".format('a',a.op.type))
b = tf.get_variable(name='b', shape=[5,10], initializer=tf.initializers.random_normal(), dtype=tf.float32)
print("\'{}\' is a {}".format('b',b.op.type))
c = tf.convert_to_tensor(np.ones(shape=(5,10),dtype=np.float32))
print("\'{}\' is a {}".format('c',c.op.type))
d = tf.add
print("\'{}\' is a {}".format('d',d))

'a' is a Placeholder
'b' is a VariableV2
'c' is a Const
'd' is a <function add at 0x0000022E95AC8620>


In [3]:
# tf_c => placeholder (shape 2x1, dtype float32)
# tf_d => variable (shape 3x2, dtype float32)
# tf_op => tf.matmul(tf_d, tf_c)
tf.reset_default_graph()

tf_c = tf.placeholder(name='c', shape=[2,1], dtype=tf.float32)
tf_d = tf.get_variable(
    name='d', shape=[3,2], dtype=tf.float32,
    initializer=tf.random_normal_initializer()
                      )
tf_op = tf.matmul(tf_d, tf_c)

### 1.1.1. Using placeholders

In [5]:
tf.reset_default_graph()

# TODO: Define a placeholder tf_a which has shape [1] and type float32

tf_add_ab = tf.add(tf_a, 4, name='add')

with tf.Session() as sess:    
    for a in [[1],[2],[3]]:
        a_plus_b = sess.run(tf_add_ab, feed_dict={tf_a:a})
        print("a ({}) + 4 = {}".format(a, a_plus_b))

a ([1]) + 4 = [5.]
a ([2]) + 4 = [6.]
a ([3]) + 4 = [7.]


### 1.1.2. Using variables

In [6]:
tf.reset_default_graph()

# TODO: Define a variable tf_a which has shape [1], type float32 and initialized with constant_initializer having value 2

tf_update_a = tf.assign(tf_a, [3])
init = tf.global_variables_initializer()
with tf.Session() as sess:    
    sess.run(init)
    a = sess.run(tf_a)
    a_updated = sess.run(tf_update_a)
    print("a = ({}) (before updating)".format(a))
    print("a = ({}) (after updating)".format(a_updated))

a = ([2.]) (before updating)
a = ([3.]) (after updating)


### 1.1.3. Using operations

In [4]:
tf.reset_default_graph()
with tf.Session() as sess:    
    
    # TODO: Define an operation that adds 3 and 5 together and assign it to tf_a_plus_b
    
    a = sess.run(tf_a_plus_b)
    print("a = {}".format(a))

a = 8


## How do we run a Python program

When you write a normal python program it is *defined-by-run*, meaning the lines will be executed immediately as they are called by the interpreter. And if you have any loops changing a single variable within the loop, the variable will be overidden by the value produced in the current iteration.

In [5]:
saving_per_week = 1000
dreams = [("House", 1000000), ("Car", 50000), ("Desktop-DL", 5000)]

# TODO: For each dream compute how many weeks you need to save money
for item, price in dreams:
    num_week = price/saving_per_week
    print(
        'You have to save {} weeks in order to by a {}'.format(
        int(num_week), item))

You have to save 1000 weeks in order to by a House
You have to save 50 weeks in order to by a Car
You have to save 5 weeks in order to by a Desktop-DL


In [6]:
print([v for v in dir() if (not v.startswith('_')) and (not v.startswith('tf')) and v!='In' and v!='Out'])

['dreams', 'exit', 'get_ipython', 'item', 'np', 'num_week', 'price', 'quit', 'saving_per_week']


## Let's try this with TensorFlow

In [10]:
tf.reset_default_graph()

saving_per_week = 1000
dreams = [("House", 1000000), ("Car", 50000), ("Desktop-DL", 5000)]

# TODO: Use tf.get_variable and tf ops to compute how many weeks need to be saved for each dream
with tf.Session() as sess:
    for item, price in dreams:
        # tf_num_weeks => tf.get_variable
        tf_num_weeks = tf.Variable(
            price/saving_per_week,
            name='num_weeks', dtype=tf.float32
        )
        
        tf.global_variables_initializer().run()
        
        num_weeks = sess.run(tf_num_weeks)
        print(
        'You have to save {} weeks in order to by a {}'.format(
        int(num_weeks), item))
        

You have to save 1000 weeks in order to by a House
You have to save 50 weeks in order to by a Car
You have to save 5 weeks in order to by a Desktop-DL


In [11]:
print("Variables in the graph")
print([v.name for v in tf.global_variables()])
print("\nOperations in the graph")
print([op.name for op in tf.get_default_graph().get_operations()])

Variables in the graph
['num_weeks:0', 'num_weeks_1:0', 'num_weeks_2:0']

Operations in the graph
['num_weeks/initial_value', 'num_weeks', 'num_weeks/Assign', 'num_weeks/read', 'init', 'num_weeks_1/initial_value', 'num_weeks_1', 'num_weeks_1/Assign', 'num_weeks_1/read', 'init_1', 'num_weeks_2/initial_value', 'num_weeks_2', 'num_weeks_2/Assign', 'num_weeks_2/read', 'init_2']


## Oh oooh ... a TensorFlow program is different...

A TensorFlow programs use graph based execution. That is, they first build a graph and then do computations over and over again on the graph with different inputs. A TensorFlow graph is like a [Galton board](https://en.wikipedia.org/wiki/Bean_machine). Your input can change (the small balls) but the screws/pegs stay fixed. And depending on how you drop the ball, the output (i.e. the bin it falls to) will be differnt.

![Galton board](../images/galton_board.png)

## 1.2. Writing a TensorFlow program

In [15]:
tf.reset_default_graph()

saving_per_week = 1000
dreams = [("House", 1000000), ("Car", 50000), ("Desktop-DL", 5000)]

with tf.Session() as sess:
    """ Defining Graph """
    
    # TODO: Define a placeholder tf_price that has shape None, type int32 and name tf_price
    tf_price = tf.placeholder(
        name='price', shape=[], dtype=tf.float32)
    tf_x = tf.placeholder()
    # TODO: Define an operation divide which divides tf_price by saving_per_week and call it num_weeks, and assign to tf_num_weeks
    tf_num_weeks = tf_price/saving_per_week
    
    """ Executing the Graph """
    sess.run(tf.global_variables_initializer())
    #tf.global_variables_initializer().run()
    
    for item, price in dreams:
        
        # TODO: Get the number of weeks by executing tf_num_weeks with correct value for tf_price placeholder and assign it to "weeks"
        weeks, new_price = sess.run(
            [tf_num_weeks,tf_price], 
            feed_dict={tf_price:price, tf_x:xx}
        )
        print("You have to save {:d} weeks in order to by a {},{}".format(
            int(weeks), item, new_price)
             )

You have to save 1000 weeks in order to by a House,1000000.0
You have to save 50 weeks in order to by a Car,50000.0
You have to save 5 weeks in order to by a Desktop-DL,5000.0


## Quick Trivia: What would be the following variable's name in the graph

In [2]:
a = tf.get_variable('b', initializer=5)

## 1.3. Scoping variables

Scoping is context managing for variables. With scopes you are essentially putting variables in different scopes, and retrieve them from the correct scope when you need them.

### Who let those variables out?

In [16]:
tf.reset_default_graph()

# TODO: Create tf_n_beds (3), tf_area (100), tf_has_park (1) variables with the initial value within brackets
#       using tf.Variable. Then compute the house price using
#       10000* tf_n_beds + 5000 * tf_area + 50000 * tf_has_park
def compute_house_price():
    """ Computing house price using number of beds, area and has a park"""
    tf_n_beds = tf.Variable(3, name='n_beds')
    tf_area = tf.Variable(100, name='area')
    tf_has_park = tf.Variable(1, name='has_park')
    
    return 10000*tf_n_beds + \
            5000 * tf_area + \
            50000 * tf_has_park

with tf.Session() as sess:
    
    tf_house_price = compute_house_price()
    tf.global_variables_initializer().run()
    print('My house price is {}'.format(sess.run(tf_house_price)))
    print("\nVariables in the graph")
    print([v.name for v in tf.global_variables()])

My house price is 580000

Variables in the graph
['n_beds:0', 'area:0', 'has_park:0']


In [17]:
tf.reset_default_graph()

with tf.Session() as sess:
    
    for _ in range(10):

        # TODO: call the compute_house_price 10 times in this loop
        tf_house_price = compute_house_price()
        tf.global_variables_initializer().run()
        print('My house price is {}'.format(sess.run(tf_house_price)))
        
    print("\nVariables in the graph")
    
    #TODO: Print the variables in the graph
    print([v.name for v in tf.global_variables()])

My house price is 580000
My house price is 580000
My house price is 580000
My house price is 580000
My house price is 580000
My house price is 580000
My house price is 580000
My house price is 580000
My house price is 580000
My house price is 580000

Variables in the graph
['n_beds:0', 'area:0', 'has_park:0', 'n_beds_1:0', 'area_1:0', 'has_park_1:0', 'n_beds_2:0', 'area_2:0', 'has_park_2:0', 'n_beds_3:0', 'area_3:0', 'has_park_3:0', 'n_beds_4:0', 'area_4:0', 'has_park_4:0', 'n_beds_5:0', 'area_5:0', 'has_park_5:0', 'n_beds_6:0', 'area_6:0', 'has_park_6:0', 'n_beds_7:0', 'area_7:0', 'has_park_7:0', 'n_beds_8:0', 'area_8:0', 'has_park_8:0', 'n_beds_9:0', 'area_9:0', 'has_park_9:0']


### With Scoping
Scoping provides you protection against creating redundant variables

In [18]:
tf.reset_default_graph()

# TODO: Create tf_n_beds (3), tf_area (100), tf_has_park (1) variables with the initial value within brackets
# using tf.get_variable (use initializer keyword argument to specify the initial value). 
# Then compute the house price using
# 10000* tf_n_beds + 5000 * tf_area + 50000 * tf_has_park
def compute_house_price():
    """ Computing house price using number of beds, area and has a park"""
    tf_n_beds = tf.get_variable(
        'n_beds', dtype=tf.float32, shape=[],
        initializer=tf.constant_initializer(3)
    )
    tf_area = tf.get_variable(
        'area', dtype=tf.float32, shape=[],
        initializer=tf.constant_initializer(100)
    )
    tf_has_park = tf.get_variable(
        'has_park', dtype=tf.float32, shape=[],
        initializer=tf.constant_initializer(1)
    )
    return 10000*tf_n_beds + 5000 * tf_area + 50000 * tf_has_park

with tf.Session() as sess:
    
    for _ in range(10):
        with tf.variable_scope('house', reuse=tf.AUTO_REUSE):
            # TODO: Call the compute_house_price function within the scope
            tf_house_price = compute_house_price()
        tf.global_variables_initializer().run()
        print('My house price is {}'.format(sess.run(tf_house_price)))
    print("\nVariables in the graph")
    print([v.name for v in tf.global_variables()])

My house price is 580000.0
My house price is 580000.0
My house price is 580000.0
My house price is 580000.0
My house price is 580000.0
My house price is 580000.0
My house price is 580000.0
My house price is 580000.0
My house price is 580000.0
My house price is 580000.0

Variables in the graph
['house/n_beds:0', 'house/area:0', 'house/has_park:0']


In [2]:
tf.reset_default_graph()

with tf.variable_scope('layer_1'):
    tf.get_variable('w', shape=[5,10], initializer=tf.initializers.random_normal(), dtype=tf.float32)
    tf.get_variable('b', shape=[5,10], initializer=tf.initializers.random_normal(), dtype=tf.float32)
    
with tf.variable_scope('layer_2'):
    tf.get_variable('w', shape=[5,10], initializer=tf.initializers.random_normal(), dtype=tf.float32)
    tf.get_variable('b', shape=[5,10], initializer=tf.initializers.random_normal(), dtype=tf.float32)

for v in tf.global_variables(): 
    print('Variable \'{}\' exists.'.format(v.name))

Variable 'layer_1/w:0' exists.
Variable 'layer_1/b:0' exists.
Variable 'layer_2/w:0' exists.
Variable 'layer_2/b:0' exists.


In [7]:
tf.reset_default_graph()

layers = [{"name":"layer1", "shape":[5,10]}, 
          {"name":"layer2", "shape":[10,3]}]

for layer in layers:

    with tf.variable_scope(layer["name"], reuse=tf.AUTO_REUSE):
        w = tf.get_variable(
            'w', shape=layer["shape"], initializer=tf.constant_initializer())
        b = tf.get_variable(
            'b', shape=layer["shape"][-1], initializer=tf.constant_initializer())

print(tf.global_variables())
for v in tf.global_variables(): 
    print('Variable \'{}\' exists.'.format(v.name))    

{'shape': [5, 10], 'name': 'layer1'}
<tf.Variable 'layer1/w:0' shape=(5, 10) dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)> <tf.Variable 'layer1/b:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>
{'shape': [10, 3], 'name': 'layer2'}
<tf.Variable 'layer2/w:0' shape=(10, 3) dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)> <tf.Variable 'layer2/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>
[]


## 1.4 What happens if we don't use `sess.run`

In [19]:
tf.reset_default_graph()
# TODO: Create a variable tf_a using 
# tf.get_variable of shape [1] and initialize with 
# tf.constant_initializer
tf_a = tf.get_variable('a', shape=[1], initializer=tf.constant_initializer)
# TODO: use tf.add operation to add 2 to tf_a and 
# print the result
tf_out = tf.add(tf_a, 2)
print(tf_out)

Tensor("Add:0", shape=(1,), dtype=float32)


## 1.5. Eager execution
Eager execution let's you to run things without building a computational graph

In [ ]:
# Restarting the Kernel 
# (Eager execution needs to be the very first thing called when a kernel stands up)
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
import tensorflow as tf
try:
    tf.enable_eager_execution()
except:
    print("Is eager execution already running? {}".format(tf.executing_eagerly()))
    
import numpy as np

In [2]:
tf.reset_default_graph()
# TODO: Create a variable tf_a using 
# tf.get_variable of shape [1] and initialize with 
# tf.constant_initializer
tf_a = tf.get_variable('a', shape=[1], initializer=tf.constant_initializer)
# TODO: use tf.add operation to add 2 to tf_a and 
# print the result
tf_out = tf.add(tf_a, 2)
print(tf_out)

tf.Tensor([2.], shape=(1,), dtype=float32)
